In [1]:
import numpy as np
import numpy.linalg as la
import scipy.optimize as scop
import os
from os import listdir
from os.path import isfile, join

from scipy.sparse import csc_matrix, kron, identity
import scipy.sparse.linalg as spal

In [2]:
def newton(A, B, n):
    zrs = np.zeros((n,n))
    k = 1 / (1 + la.norm(B, 1) * la.norm(A, np.inf))  
    print("coefficient = ", k)
    equation = lambda X: k * (np.matmul(np.matmul(A, X), B) + B) - X
    sol = scop.newton_krylov(equation, zrs, verbose = True, f_tol = pow(10, - 13), maxiter = 15)
    %timeit -r 1 -n 1 scop.newton_krylov(equation, zrs, verbose = True, f_tol = pow(10, - 13), maxiter = n * n // 4)
    return ((sol != 0).astype(int))

In [3]:
def parse_graph(_file):
    nc = 0
    with open(_file, 'r') as graph:
        for line in graph:
            terms = line.split()
            nc = max(nc, int(terms[0]), int(terms[2]))
    nc += 1
    
    A = np.zeros((nc,nc))
    B = np.zeros((nc,nc))
    with open(_file, 'rt') as graph:
        for line in graph:
            terms = line.split()
            if terms[1] == "SCOR":
                A[int(terms[0])][int(terms[2])] = 1
            elif terms[1] == "SCO":
                B[int(terms[0])][int(terms[2])] = 1
                
    print("As = ", np.sum(A), "Bs = ", np.sum(B))
    return A, B, nc

In [4]:
def main(graphfile, resultfile):
    A, B, n = parse_graph(graphfile)
    print("verteces_count: ", n)

    res = newton(A, B, n)
    resf = open(resultfile, 'w')
    count = 0
    for i in range(n):
        for j in range(n):
            if res[i][j] == 1: 
                count += 1
            resf.write("%d " %res[i][j])
        resf.write("\r\n")
    resf.close()
    print("count = ", count)

In [5]:
gr_path = "/home/ju-sus/Documents/CFPQ/data/RDF-graphs"
res_path = "/home/ju-sus/Documents/CFPQ/results"
graphs = [f for f in listdir(gr_path) if isfile(join(gr_path, f))]
print(graphs)
results = ["res-" + f for f in graphs]
print(results)

['people_pets.txt', 'atom-primitive.txt', 'skos.txt', 'core.txt', 'generations.txt', 'biomedical-mesure-primitive.txt', 'pizza.txt', 'univ-bench.txt', 'travel.txt', 'foaf.txt', 'wine.txt', 'funding.txt']
['res-people_pets.txt', 'res-atom-primitive.txt', 'res-skos.txt', 'res-core.txt', 'res-generations.txt', 'res-biomedical-mesure-primitive.txt', 'res-pizza.txt', 'res-univ-bench.txt', 'res-travel.txt', 'res-foaf.txt', 'res-wine.txt', 'res-funding.txt']


In [6]:
for i in range(len(graphs)):
    print()
    print(graphs[i])
    graph = os.path.join(gr_path, graphs[i])
    result = os.path.join(res_path, results[i])
    %timeit -r 1 -n 1 main(graph, result)


people_pets.txt
As =  33.0 Bs =  33.0
verteces_count:  337
coefficient =  0.2
0:  |F(x)| = 2.4159e-09; step 1
1:  |F(x)| = 5.55112e-17; step 1
0:  |F(x)| = 2.4159e-09; step 1
1:  |F(x)| = 5.55112e-17; step 1
74.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
count =  37
356 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

atom-primitive.txt
As =  122.0 Bs =  122.0
verteces_count:  291
coefficient =  0.5
0:  |F(x)| = 5.62119e-09; step 1
1:  |F(x)| = 5.55112e-17; step 1
0:  |F(x)| = 5.62119e-09; step 1
1:  |F(x)| = 5.55112e-17; step 1
55.2 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
count =  122
260 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

skos.txt
As =  1.0 Bs =  1.0
verteces_count:  144
coefficient =  0.5
0:  |F(x)| = 0; step 1
0:  |F(x)| = 0; step 1
6.33 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
count =  1
62.3 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

core.txt
As =  178.0 Bs =  178.0
v